Pre-processing of part of the ketamine data, the .mat file seems to contain some wrong information (states only 701 samples per electrodes have been collected)

The original data is saved in .mat (description) and .dat (c

Until now only locally, because collab isnt collaborating

In [ ]:
import os
import numpy as np
from scipy.io import loadmat
import pandas as pd

# Define the directory path
data_dir = r"C:\Users\s2364956\Desktop\Data\ComplexSystems\Ketamine\Depressed\LTP_imperial"

# Initialize lists to store data and metadata
all_eeg_data = []  # For 3D array (n_files × n_channels × n_samples)
file_names = []
channel_names = None
fsample = None

# Loop through all .mat files in the directory
for mat_file in sorted(os.listdir(data_dir)):
    if mat_file.endswith('.mat'):
        mat_path = os.path.join(data_dir, mat_file)

        # Load .mat file
        cmeta = loadmat(mat_path)
        D = cmeta['D'][0, 0]

        # Extract metadata (only once, assuming consistency across files)
        if channel_names is None:
            n_channels = len(D['channels'][0])
            channel_names = [str(ch[0]) for ch in D['channels'][0]]
            fsample = int(D['Fsample'][0][0])

        # Load paired .dat file
        dat_path = mat_path.replace('.mat', '.dat')
        continuous_data = np.fromfile(dat_path, dtype='float32')
        n_samples_total = continuous_data.size // n_channels
        eeg_data = continuous_data.reshape((n_channels, n_samples_total), order='F')

        # Store in list
        all_eeg_data.append(eeg_data)
        file_names.append(mat_file)
        print(f"Processed: {mat_file} | Shape: {eeg_data.shape}")

# Stack all files into a 3D array (n_files × n_channels × n_samples)
# Note: Assumes all files have the same n_samples_total! If not, pad/truncate.
min_samples = min(arr.shape[1] for arr in all_eeg_data)  # Use smallest n_samples if inconsistent
eeg_3d = np.stack([arr[:, :min_samples] for arr in all_eeg_data])

print("\n3D Array Shape (Files × Channels × Samples):", eeg_3d.shape)

Processed: FullcbeffMspmeeg_KETDEP0017_Day1_LTPPre_ICACleaned.mat | Shape: (64, 504720)
Processed: FullcbeffMspmeeg_KETDEP002_Day1_LTPPre1_ICACleaned.mat | Shape: (64, 504720)
Processed: FullcbeffMspmeeg_KETDEP003_Day2_LTPPre_ICACleaned.mat | Shape: (64, 504720)
Processed: FullcbeffMspmeeg_KETDEP004_Day2_LTPPre_ICACleaned.mat | Shape: (64, 504720)
Processed: FullcbeffMspmeeg_KETDEP005_Day1_LTPPre_ICACleaned.mat | Shape: (64, 504720)
Processed: FullcbeffMspmeeg_KETDEP008_Day1_LTPPre_ICACleaned.mat | Shape: (64, 504720)
Processed: FullcbeffMspmeeg_KETDEP009_Day1_LTPPre1_ICACleaned.mat | Shape: (64, 499813)
Processed: FullcbeffMspmeeg_KETDEP010_Day2_LTPPre_ICACleaned.mat | Shape: (64, 556594)
Processed: FullcbeffMspmeeg_KETDEP011_Day2_LTPPre_ICACleaned.mat | Shape: (64, 504720)
Processed: FullcbeffMspmeeg_KETDEP013_Day2_LTPPre_ICACleaned.mat | Shape: (64, 504720)
Processed: FullcbeffMspmeeg_KETDEP014_Day1_LTPPre_ICACleaned.mat | Shape: (64, 504720)
Processed: FullcbeffMspmeeg_KETDEP014_Da

In [ ]:
# create the time vector based on the minimum samples and sampling rate
time_vector = np.arange(min_samples) / fsample  # Time in seconds

# Now select the first file's data and create a DataFrame
first_file_data = eeg_3d[0]  # Shape: (n_channels, min_samples)

# Create DataFrame for first 5 time points
df_first_5 = pd.DataFrame(
    first_file_data[:, :5].T,  # Transpose to (5 samples × n_channels)
    columns=channel_names,
    index=[f"{t:.4f}s" for t in time_vector[:5]]  # Format time labels
)

# Set display options
pd.set_option('display.max_columns', 8)  # Show 8 channels at a time
pd.set_option('display.width', 1000)    # Prevent column wrapping

print(f"First 5 time points for file: {file_names[0]}")
print(f"Sampling rate: {fsample} Hz")
display(df_first_5)

First 5 time points for file: FullcbeffMspmeeg_KETDEP0017_Day1_LTPPre_ICACleaned.mat
Sampling rate: 1000 Hz


,['Fp1'],['Fp2'],['F7'],['F3'],...,['PO3'],['POz'],['PO4'],['PO8']
0.0000s,-6.419561,-5.155789,-7.589287,-2.592353,...,5.220371,1.396340,0.635933,1.679263
0.0010s,-6.526850,-5.101662,-7.303430,-2.576094,...,5.133623,1.251419,0.655041,1.749535
0.0020s,-6.652922,-5.005090,-7.013856,-2.586167,...,5.023960,1.096253,0.677728,1.816788
0.0030s,-6.795858,-4.869094,-6.728922,-2.621862,...,4.894469,0.934003,0.703041,1.883018
0.0040s,-6.954254,-4.697541,-6.457341,-2.682146,...,4.748647,0.768209,0.730564,1.951272
